In [3]:
import os
import re
import json
import sqlite3
import subprocess
import webbrowser
from collections import Counter
import tkinter as tk
from tkinter import scrolledtext, ttk, Canvas
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import spacy
from langdetect import detect
import torch
import faiss
import heapq
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import logging
import warnings
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import logging as transformers_logging
from keybert import KeyBERT
import pyperclip

# === INITIALISATION ===

# Chargement des variables de configuration

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
config_path = os.path.join(PROJECT_ROOT, "config.json")

def expand_path(value):
    expanded = os.path.expanduser(value)
    if not os.path.isabs(expanded):
        expanded = os.path.normpath(os.path.join(PROJECT_ROOT, expanded))
    return expanded

def load_config(config_path):
    with open(config_path, "r", encoding="utf-8") as f:
        raw_config = json.load(f)

    path_keys = {
        "venv_activate_path",
        "lmstudio_folder_path",
        "sync_script_path",
        "project_script_path",
        "db_path",
        "stopwords_file_path"
    }

    config = {}
    for key, value in raw_config.items():
        if isinstance(value, str):
            if key == "summarizing_model" and value == "model/barthez-orangesum-abstract":
                config[key] = expand_path(value)
            elif key in path_keys:
                config[key] = expand_path(value)
            else:
                config[key] = value
        else:
            config[key] = value
    return config

config = load_config(config_path)

stopwords_path = config.get("stopwords_file_path", "stopwords_fr.json")
with open(stopwords_path, "r", encoding="utf-8") as f:
    french_stop_words = set(json.load(f))

combined_stopwords = ENGLISH_STOP_WORDS.union(french_stop_words)

# Masquage des avertissements
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#logging.getLogger("transformers").setLevel(logging.ERROR)
#logging.getLogger("torch").setLevel(logging.ERROR)
#torch._C._log_api_usage_once = lambda *args, **kwargs: None
#warnings.filterwarnings("ignore", message="Unfeasible length constraints", category=UserWarning, module="transformers.generation.utils")

# Connexion à la base SQLite
conn = sqlite3.connect(config["db_path"])
cur = conn.cursor()

# Initialisation des modèles
nlp_fr = spacy.load("fr_core_news_lg")
nlp_en = spacy.load("en_core_web_lg")
kw_model = KeyBERT()
summarizing_model = config.get("summarizing_model", "model/barthez-orangesum-abstract")
summarizing_pipeline = pipeline(task="summarization", model=summarizing_model, framework="pt")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Index vectoriel
VECTOR_DIM = 384
faiss_index = faiss.IndexFlatL2(VECTOR_DIM)


Device set to use mps:0


In [4]:
# === FONCTIONS PRINCIPALES ===

# Synchronisation des conversations
def sync_conversations(config, label_status):
    sync_path = config.get("sync_script_path")
    if not sync_path:
        label_status.config(text="sync_script_path introuvable.")
        return False
    try:
        subprocess.run(["python3", sync_path], check=True)
        label_status.config(text="Synchronisation terminée.")
        return True
    except subprocess.CalledProcessError:
        label_status.config(text="Erreur lors de la synchronisation.")
        return False
    except FileNotFoundError:
        label_status.config(text="Script de synchronisation introuvable.")
        return False

# Pércuteur
def on_ask():
    question = entry_question.get("1.0", "end-1c")
    if not question.strip():
        update_status("⚠️ Merci de saisir une question.", error=True)
        return
    update_status("⚙️ Traitement en cours...")
    root.update()
    try:
        context = get_relevant_context(question, limit=context_count_var.get()) #", limit=context_count_var.get()" ajoutée slider contexte
        prompt = generate_prompt_paragraph(context, question)
        pyperclip.copy(prompt)
        text_output.delete('1.0', tk.END)
        text_output.insert(tk.END, prompt)
        
        # Calcul des métriques
        context_count = len(context)
        token_count = len(prompt.split())
        
        update_status(
            f"Prompt généré ({token_count} tokens) | Contexte utilisé : {context_count} éléments",
            success=True
        )
    except Exception as e:
        update_status(f"❌ Erreur : {str(e)}", error=True)

In [8]:
# === CONTEXTE ===

# Choix NLP selon langue
def get_nlp_model(text):
    try:
        lang = detect(text)
    except:
        lang = "fr"  # défaut français si détection impossible
    
    if lang.startswith("en"):
        return nlp_en
    else:
        return nlp_fr


# Récupération des mots-clés de la question initiale
root = tk.Tk()
keyword_count_var = tk.IntVar(value=5)
context_count_var = tk.IntVar(value=3)
multiplier = config.get("keyword_multiplier", 2)

def extract_keywords(text, top_n=None):
    if top_n is None:
        top_n = keyword_count_var.get()

    # Extraction brute avec KeyBERT
    raw_keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 1),
        stop_words=list(combined_stopwords),
        top_n=top_n * multiplier)

    stopwords_set = set(combined_stopwords)

    tokens = re.findall(r'\b[a-zA-Z\-]{3,}\b', text.lower())
    token_freq = Counter([tok for tok in tokens if tok not in stopwords_set])

    # Fonction de validation rapide des mots clés
    def is_valid_kw(kw):
        return (
            kw not in stopwords_set and
            len(kw) > 2 and
            kw.isalpha() or '-' in kw
        )

    # Tri par fréquence dans le texte #  /!\ ==peu entrainer des doublons== /!\
    filtered_raw = []
    for kw, weight in raw_keywords:
        kw_clean = kw.lower().strip()
        if is_valid_kw(kw_clean):
            freq = token_freq.get(kw_clean, 0)
            filtered_raw.append((freq, kw_clean, weight))

    top_filtered = heapq.nlargest(top_n, filtered_raw, key=lambda x: x[0])

    seen = set()
    filtered_keywords = []
    for freq, kw_clean, weight in top_filtered:
        if kw_clean not in seen:
            seen.add(kw_clean)
            filtered_keywords.append((kw_clean, weight, freq))

    return filtered_keywords

def get_vector_for_text(text):
    print(f"Type de embedding_model : {type(embedding_model)}")
    print(f"embedding_model : {embedding_model}")
    print(f"Is instance of SentenceTransformer? {isinstance(embedding_model, SentenceTransformer)}")

    vec = embedding_model.encode([text])
    return np.array(vec[0], dtype='float32')

# Récupération des anciennes conversations pertinentes

def get_relevant_context(user_question, limit=None):
    if limit is None:
        limit = context_count_var.get()

    keywords = extract_keywords(user_question)
    if not keywords or not isinstance(keywords, (list, tuple)):
        print("Warning: keywords non valides ou vides:", keywords)
        return []

    # S’assurer que keywords est une liste de tuples (keyword, score) ou similaire
    try:
        keyword_strings = [kw[0] for kw in keywords if isinstance(kw, (list, tuple)) and len(kw) > 0]
    except Exception as e:
        print(f"Erreur extraction keywords: {e}")
        return []

    if not keyword_strings:
        print("Warning: liste keyword_strings vide après extraction")
        return []

    placeholders = ', '.join(['?'] * len(keyword_strings))

    # 1. Récupérer les conversations + vecteurs filtrés par mots-clés
    query_vectors = f'''
        SELECT conversation_id, vector
        FROM vectors
        WHERE keyword IN ({placeholders})
    '''
    cur.execute(query_vectors, keyword_strings)
    vector_rows = cur.fetchall()  # [(conversation_id, vector_str), ...]

    if not vector_rows:
        return []

    convo_ids = []
    vectors = []

    # Convertir les vecteurs string en numpy array
    for convo_id, vec_str in vector_rows:
        print(f"Vecteur brut convo {convo_id}:", repr(vec_str))
        if not vec_str or not vec_str.strip():
            print(f"Attention: vecteur vide pour conversation {convo_id}")
            continue
        try:
            vec_str_clean = vec_str.strip().replace('\n', '').replace('\r', '').replace('[', '').replace(']', '')
            vec = np.fromstring(vec_str_clean, sep=',').astype('float32')
        except Exception as e:
            print(f"Erreur conversion vecteur pour convo {convo_id}: {e}")
            continue
        convo_ids.append(convo_id)
        vectors.append(vec)

    if not vectors:
        return []

    vectors = np.array(vectors).astype('float32')

    # Normalisation des vecteurs pour cosine similarity
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    vectors = vectors / np.clip(norms, a_min=1e-10, a_max=None)  # éviter division par 0

    # Construire l'index FAISS pour Inner Product (cosine similarity)
    faiss_index = faiss.IndexFlatIP(vectors.shape[1])
    faiss_index.add(vectors)

    # Vecteur de la question
    question_vec = get_vector_for_text(user_question).astype('float32').reshape(1, -1)

    # Normaliser le vecteur question aussi
    question_norm = np.linalg.norm(question_vec, axis=1, keepdims=True)
    question_vec = question_vec / np.clip(question_norm, a_min=1e-10, a_max=None)

    # Recherche des k voisins les plus proches (cosine similarity)
    distances, indices = faiss_index.search(question_vec, limit)

    # Récupérer les convo_ids correspondants
    found_convo_ids = [convo_ids[idx] for idx in indices[0] if idx != -1]

    if not found_convo_ids:
        return []

    placeholders_ids = ', '.join(['?'] * len(found_convo_ids))

    # Récupérer les conversations
    query_contexts = f'''
        SELECT id, user_input, llm_output, timestamp
        FROM conversations
        WHERE id IN ({placeholders_ids})
    '''
    cur.execute(query_contexts, found_convo_ids)
    context_rows = cur.fetchall()

    # Récupérer les mots-clés associés
    query_keywords = f'''
    SELECT conversation_id, keyword
    FROM vectors
    WHERE conversation_id IN ({placeholders_ids})
    '''
    cur.execute(query_keywords, found_convo_ids)
    keyword_rows = cur.fetchall()

    print("keyword_rows (after fetch):", keyword_rows)

    # Test plus sûr sur les données
    keywords_by_convo = {}
    for row in keyword_rows:
        if isinstance(row, (list, tuple)):
            if len(row) == 2:
                convo_id, keyword = row
                keywords_by_convo.setdefault(convo_id, set()).add(keyword)
            else:
                print(f"Unexpected row length: {row}")
        else:
            print(f"Unexpected row type (expect tuple/list): {type(row)} - value: {row}")


    # Construire le résultat final avec les scores de similarité
    filtered_context = []
    for idx, (convo_id, user_input, llm_output, timestamp) in enumerate(context_rows):
        kws = list(keywords_by_convo.get(convo_id, []))
        score = distances[0][idx] if idx < len(distances[0]) else 0.0  # sécuriser accès score
        filtered_context.append((user_input, llm_output, timestamp, kws, score))

    return filtered_context



# Nettoyage du texte
def nlp_clean_text(text, max_chunk_size=500):
    text = re.sub(r'```(?:python)?\s*.*?```', '', text, flags=re.DOTALL)
    nlp = get_nlp_model(text)
    chunks, current_chunk, current_length = [], [], 0

    for sent in nlp(text).sents:
        s = sent.text.strip()
        if len(s) < 20:
            continue
        if current_length + len(s) < max_chunk_size:
            current_chunk.append(s)
            current_length += len(s)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [s]
            current_length = len(s)
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return " ".join(chunks[:3])  # limite à 3 blocs maximum

In [5]:
# === CONSTRUCTION DU PROMPT ===

# Compression du contexte extrait
def summarize(text, focus_terms=None, max_length=1024):
    transformers_logging.set_verbosity_error()
    try:
        # Filtrage des phrases importantes si focus_terms donné
        if focus_terms:
            sentences = [s for s in text.split('.') 
                        if any(term.lower() in s.lower() for term in focus_terms)]
            text = '. '.join(sentences)[:2000] or text[:2000]

        # Résumé avec le texte filtré
        result = summarizing_pipeline(
            text,
            max_length=max_length,
            min_length=max_length // 2,
            no_repeat_ngram_size=3,
            do_sample=False,
            truncation=True
        )
        return nlp_clean_text(result[0]['summary_text'])

    except Exception as e:
        print(f"Erreur summarization : {e}")
        return text[:max_length] + "... [résumé tronqué]"
    
# Construction du prompt
def generate_prompt_paragraph(context, question, target_tokens=1000):
    if not context:
        return f"{question}"

    # 1. Prétraitement
    processed_items = []
    for item in context[:3]:  # Nombre max d'éléments dans le contexte
        try:
            # Extraction sécurisée
            user_input = str(item[0])[:300]  # Troncature des questions longues
            llm_output = str(item[1])
            keyword = str(item[5]) if len(item) > 5 and str(item[3]).strip() not in {"", "none", "null", "1", "2", "3"} else None

            # Summarization, netooyage, segmentation
            summary = nlp_clean_text(summarize(llm_output))
            processed_items.append({
                'question': user_input,
                'summary': summary,
                'keyword': keyword if keyword else None
            })
        except Exception as e:
            print(f"Erreur traitement item : {e}")
            continue

    if not processed_items:
        return question

    # 2. Construction du prompt
    parts = []

    # Partie questions
    if processed_items:
        questions = [f"'{item['question']}'" for item in processed_items]
        if len(questions) == 1:
            parts.append(f"Tes discussions avec l'utilisateur t'ont amené à répondre à cette question : {questions[0]}")
        else:
            *init, last = questions
            parts.append(f"Tes discussions avec l'utilisateur t'ont amené à répondre à ces questions :  {', '.join(init)}, et enfin {last}")

    # Partie mots-clés
    keywords = {item['keyword'] for item in processed_items if item['keyword']}
    if keywords:
        parts.append(f"Mots-clés pertinents : {', '.join(sorted(keywords))}")

    # Partie résumés
    if processed_items:
        summaries = [f"- {item['summary']}" for item in processed_items]
        parts.append("Ces intéractions vous ont amené à discuter de ces sujets :\n" + "\n".join(summaries))

    # Question actuelle
    parts.append(f"Réponds maintenant à cette question, dans le contexte de vos discussions précédentes : {question}")

    return "\n".join(parts)

In [ ]:
# === TEST RAPIDE NO GUI ===
question = 'Comment la photocatalyse rédox est actuellement appliquée en "drug discovery" du secteur pharmaceutique ?'
context = get_relevant_context(question)
prompt = generate_prompt_paragraph(context, question)
print(prompt)


Vecteur brut convo 42: '[-0.03203989565372467, 0.007134939078241587, -0.07414238899946213, -0.04461883753538132, -0.11072579026222229, -0.006147498730570078, 0.0804787278175354, 0.10960470885038376, 0.014741298742592335, -0.045208513736724854, -0.005541444756090641, -0.02903510071337223, 0.045439448207616806, 0.045019082725048065, -0.07905307412147522, -0.0662766695022583, 0.029352720826864243, -0.003597831819206476, 0.05020849034190178, 0.036834850907325745, -0.08112623542547226, 0.06656025350093842, 0.03899535909295082, 0.06608276069164276, -0.04336779564619064, 0.06670437753200531, -0.041366688907146454, 0.015143846161663532, -0.05393824353814125, -0.06567473709583282, 0.07326257228851318, 0.05076780170202255, 0.046099547296762466, -0.023754771798849106, 0.03979028761386871, 0.029361233115196228, -0.07884436845779419, 0.04738137871026993, 0.05724068358540535, 0.06068364530801773, -0.024577254429459572, -0.0722459927201271, -0.03671948239207268, 0.003274217713624239, 0.07636512070894

/Users/victorcarre/Code/Projects/llm-memorization/.venv/lib/python3.13/site-packages/transformers/generation/utils.py:1633: UserWarning: Unfeasible length constraints: `min_length` (512) is larger than the maximum possible length (257). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Tes discussions avec l'utilisateur t'ont amené à répondre à cette question : 'Comment faire du "drug discovery" dans le secteur pharmaceutique ?'
Ces intéractions vous ont amené à discuter de ces sujets :
- Le processus de drug discovery est un des plus importants et coûteux dans le secteur pharmaceutique. Voici les étapes clés du processus : 1. Identification des cibles : La première étape consiste à identifier les protéines ou les mécanismes cellulaires impliqués dans une maladie donnée ; ensuite, les scientifiques développent des molécules qui interagissent avec la cible et modulent son activité (chimiques, biologiques, cytogènes, etc). Synthèse de composés : Les molécules conçues sont then synthétisées et testées en laboratoire pour déterminer leur efficacité et leur sécurité. Approbation réglementaire : Si un médicament réussit ses essais cliniques, l'entreprise qui le développe doit soumettre une demande de ses filiales à une agence réglementaire, telle que la FDA en Europe ou l'

: 

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_vector_for_text(text):
    vec = embedding_model([text])
    return np.array(vec[0], dtype='float32')

test_text = "Test sentence for embedding"
print(get_vector_for_text(test_text).shape)


AttributeError: 'list' object has no attribute 'items'

In [ ]:
# === INTERFACE TKINTER ===

def update_status(message, error=False, success=False):
    """Met à jour le label de statut avec style approprié"""
    label_status.config(text=message)
    if error:
        label_status.config(foreground='#ff6b6b')
    elif success:
        label_status.config(foreground='#599258')
    else:
        label_status.config(foreground='white')

def open_github(event):
    webbrowser.open_new("https://github.com/victorcarre6")

def show_help():
    help_text = (
        "LLM Memorization and Prompt Enhancer — Aide\n\n"
        "• Synchroniser les conversations : ajoute les nouveaux échanges depuis LM Studio à la base de données.\n\n"
        "• Générer prompt : extrait les mots-clés de votre question, cherche des conversations similaires dans votre base SQL, puis compresse les informations avec un LLM local.\n\n"
        "Le prompt final est affiché puis automatiquement copié dans votre presse-papier !\n\n"
        "Pour en savoir plus, obtenir plus d'informations à propos d'un éventuel bloquage des scripts, ou me contacter, visitez : github.com/victorcarre6/llm-memorization"
    )
    help_window = tk.Toplevel(root)
    help_window.title("Aide")
    help_window.geometry("500x300")
    help_window.configure(bg="#323232")

    text_widget = tk.Text(help_window, wrap=tk.WORD, font=("Segoe UI", 8))
    text_widget.insert(tk.END, help_text)
    text_widget.config(state=tk.DISABLED)
    text_widget.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

    ok_button = tk.Button(help_window, text="Fermer", command=help_window.destroy)
    ok_button.pack(pady=10)

# === CONFIGURATION DE L'INTERFACE ===
root.title("LLM Memorization and Prompt Enhancer")
root.geometry("1000x325")
root.configure(bg="#323232")

# Style global unique
style = ttk.Style(root)
style.theme_use('clam')

# Configuration du style
style_config = {
    'Green.TButton': {
        'background': '#599258',
        'foreground': 'white',
        'font': ('Segoe UI', 11),
        'padding': 4
    },
    'Bottom.TButton': {
        'background': '#599258',
        'foreground': 'white',
        'font': ('Segoe UI', 9),
        'padding': 2
    },
    'Blue.TLabel': {
        'background': '#323232',
        'foreground': '#599258',
        'font': ('Segoe UI', 8, 'italic underline'),
        'padding': 0
    },
    'TLabel': {
        'background': '#323232',
        'foreground': 'white',
        'font': ('Segoe UI', 11)
    },
    'TEntry': {
        'fieldbackground': '#FDF6EE',
        'foreground': 'black',
        'font': ('Segoe UI', 11)
    },
    'TFrame': {
        'background': '#323232'
    },
    'Status.TLabel': {
        'background': '#323232',
        'font': ('Segoe UI', 11)
    },
    'TNotebook': {
        'background': '#323232',
        'borderwidth': 0
    },
    'TNotebook.Tab': {
        'background': '#2a2a2a',
        'foreground': 'white',
        'padding': (10, 4)
    },
    'Custom.Treeview': {
        'background': '#2a2a2a',
        'foreground': 'white',
        'fieldbackground': '#2a2a2a',
        'font': ('Segoe UI', 10),
        'bordercolor': '#323232',
        'borderwidth': 0,
    },
    'Custom.Treeview.Heading': {
        'background': '#323232',
        'foreground': '#599258',
        'font': ('Segoe UI', 11, 'bold'),
        'relief': 'flat'
    }
}

for style_name, app_config in style_config.items():
    style.configure(style_name, **app_config)

style.map('Green.TButton',
          background=[('active', '#457a3a'), ('pressed', '#2e4a20')],
          foreground=[('disabled', '#d9d9d9')])

style.map("TNotebook.Tab",
          background=[("selected", "#323232"), ("active", "#2a2a2a")],
          foreground=[("selected", "white"), ("active", "white")])


style.map('Bottom.TButton',
          background=[('active', '#457a3a'), ('pressed', '#2e4a20')],
          foreground=[('disabled', '#d9d9d9')])

# Widgets principaux
main_frame = ttk.Frame(root, style='TFrame')
main_frame.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Section question - Centrée
question_header = ttk.Frame(main_frame, style='TFrame')
question_header.pack(fill='x', pady=(0, 1))
ttk.Label(question_header, text="Poser la question :").pack(expand=True)

question_frame = tk.Frame(main_frame, bg="#323232")
question_frame.pack(pady=(0, 5), fill='x', expand=True)

entry_question = tk.Text(question_frame, height=4, width=80, wrap="word", font=('Segoe UI', 11))
entry_question.pack(side="left", fill="both", expand=True)

# Configuration du style pour la scrollbar
style = ttk.Style()
style.configure("Vertical.TScrollbar",
    troughcolor='#FDF6EE',
    background='#C0C0C0',
    darkcolor='#C0C0C0',
    lightcolor='#C0C0C0',
    bordercolor='#FDF6EE',
    arrowcolor='black',
    relief='flat')

scrollbar = ttk.Scrollbar(
    question_frame,
    orient="vertical",
    command=entry_question.yview,
    style="Vertical.TScrollbar"  # Application du style
)
scrollbar.pack(side="right", fill="y")

entry_question.config(yscrollcommand=scrollbar.set)
entry_question.bind("<Return>", lambda event: on_ask())

# Frame horizontale principale
control_frame = ttk.Frame(main_frame, style='TFrame')
control_frame.pack(fill='x', pady=(0, 10), padx=5)

# Sliders
slider_keywords_frame = ttk.Frame(control_frame, style='TFrame')
slider_keywords_frame.grid(row=0, column=0, sticky='w')

label_keyword_count = ttk.Label(slider_keywords_frame, text=f"Nombre de mots-clés : {keyword_count_var.get()}", style='TLabel')
label_keyword_count.pack(anchor='w')

slider_keywords = ttk.Scale(
    slider_keywords_frame,
    from_=1,
    to=15,
    orient="horizontal",
    variable=keyword_count_var,
    length=180,
    command=lambda val: label_keyword_count.config(text=f"Nombre de mots-clés : {int(float(val))}")
)
slider_keywords.pack(anchor='w')

slider_context_frame = ttk.Frame(control_frame, style='TFrame')
slider_context_frame.grid(row=0, column=1, padx=20, sticky='w')

label_contexts_count = ttk.Label(slider_context_frame, text=f"Nombre de contextes : {context_count_var.get()}", style='TLabel')
label_contexts_count.pack(anchor='w')

slider_contexts = ttk.Scale(
    slider_context_frame,
    from_=1,
    to=5,
    orient=tk.HORIZONTAL,
    variable=context_count_var,
    length=170,
    command=lambda val: label_contexts_count.config(text=f"Nombre de contextes : {int(float(val))}")
)
slider_contexts.pack(anchor='w')

# Boutons synchronisation et percuteur
button_frame = ttk.Frame(control_frame, style='TFrame')
button_frame.grid(row=0, column=2, sticky='e')

sync_button = ttk.Button(button_frame, text="Synchroniser les conversations", 
                         command=sync_conversations, style='Green.TButton')
sync_button.pack(side='left', padx=5)

btn_ask = ttk.Button(button_frame, text="Générer prompt", command=on_ask, style='Green.TButton')
btn_ask.pack(side='left', padx=5)
control_frame.grid_columnconfigure(2, weight=1)

# Zone de sortie étendable
output_expanded = tk.BooleanVar(value=False)

def toggle_output():
    """Basculer l'affichage de la zone de sortie et ajuster la taille de la fenêtre"""
    if output_expanded.get():
        text_output.pack_forget()
        toggle_btn.config(text="▼ Afficher le résultat")
        output_expanded.set(False)
        root.geometry("1000x325")
    else:
        text_output.pack(fill=tk.BOTH, expand=True, pady=(5, 0))
        toggle_btn.config(text="▲ Masquer le résultat")
        output_expanded.set(True)
        root.geometry("1000x750")

output_frame = ttk.Frame(main_frame, style='TFrame')
output_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 10))

# Bouton pour étendre/cacher
toggle_btn = ttk.Button(
    output_frame,
    text="▼ Afficher le résultat",
    command=toggle_output,
    style='Green.TButton'  # Utilise le même style que tes autres boutons
)
toggle_btn.pack(fill=tk.X, pady=(0, 5))

text_output = scrolledtext.ScrolledText(
    output_frame, 
    width=100, 
    height=20,
    font=('Segoe UI', 11), 
    wrap=tk.WORD, 
    bg="#FDF6EE", 
    fg="black", 
    insertbackground="black"
)

def show_infos():
    info_window = tk.Toplevel(root)
    info_window.title("Détails sur le prompt généré")
    info_window.geometry("750x725")
    container = tk.Frame(info_window, bg="#323232")
    container.pack(fill="both", expand=True)

    info_window.transient(root)
    info_window.grab_set()

    question = entry_question.get("1.0", tk.END).strip()
    if not question:
        update_status("⚠️ Posez une question d'abord.", error=True)
        return

    keywords = extract_keywords(question)
    if not keywords:
        update_status("⚠️ Aucun mot-clé extrait.", error=True)
        return

    filtered_context = get_relevant_context(question, limit=5)
    if not filtered_context:
        update_status("⚠️ Aucun contexte trouvé.", error=True)
        return

    notebook = ttk.Notebook(container, style="TNotebook")
    notebook.pack(fill="both", expand=True, padx=10, pady=10)

    single_tab = ttk.Frame(notebook, style="TFrame")
    notebook.add(single_tab, text="Mots-clés & Stats")

        # Calcul des fréquences dans les contextes
    full_text_context = " ".join(user_input + " " + llm_output for user_input, llm_output, _, _ in filtered_context).lower()
    token_list = re.findall(r'\b[a-zA-Z\-]{3,}\b', full_text_context)

    word_counts = {}
    for kw, _, _ in keywords:
        count = token_list.count(kw.lower())
        word_counts[kw] = count

    # --- Histogramme ---
    fig, ax = plt.subplots(figsize=(6, 4), dpi=100)
    bars = ax.bar(word_counts.keys(), word_counts.values(), color='#599258')

    ax.set_facecolor("#323232")
    fig.patch.set_facecolor("#323232")
    ax.set_title("Fréquence des mots-clés dans les contextes", color="white", fontsize=10)
    ax.set_ylabel("Occurrences", color="white", fontsize=10)
    ax.tick_params(axis='x', labelrotation=45, colors="white", labelsize=10)
    ax.tick_params(axis='y', colors="white", labelsize=10)
    for spine in ax.spines.values():
        spine.set_color('white')

    fig.tight_layout()

    canvas = FigureCanvasTkAgg(fig, master=single_tab)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=False, padx=10, pady=(10, 0))

    # --- Tableau Treeview ---
    cols = ('Mot-clé', 'Poids', 'Occurrences dans contextes')
    tree = ttk.Treeview(single_tab, columns=cols, show='headings', height=10, style='Custom.Treeview')
    for col in cols:
        tree.heading(col, text=col)
        tree.column(col, width=150)
    tree.pack(expand=False, fill='both', padx=10, pady=(10, 0))

    tree.tag_configure('oddrow', background='#2a2a2a')
    tree.tag_configure('evenrow', background='#383838')

    for i, (kw, weight, _) in enumerate(keywords):
        freq_in_context = word_counts.get(kw, 0)
        tag = 'evenrow' if i % 2 == 0 else 'oddrow'
        tree.insert('', tk.END, values=(kw, f"{weight:.3f}", freq_in_context), tags=(tag,))

    btn_copy = ttk.Button(single_tab, text="Copier mots-clés", command=lambda: (
        root.clipboard_clear(),
        root.clipboard_append(",".join([kw for kw, _, _ in keywords])),
    ), style='Bottom.TButton')
    btn_copy.pack(pady=10)

    # === Onglet 2 : Contexte ===
    context_frame = ttk.Frame(notebook, style="TFrame")
    notebook.add(context_frame, text="Contextes")

    lbl_container = tk.Frame(context_frame, bg="#323232")
    lbl_container.pack(fill="both", expand=True)

    tk.Label(lbl_container, text="Questions contextuelles :", fg="white", bg="#323232", font=("Segoe UI", 10, "bold")).pack(pady=5)
    tk.Label(lbl_container,
             text="Légende :\n- 1 mot clef : rouge\n- 2 ou 3 : orange\n- >3 : vert",
             fg="white", bg="#323232", justify="left", wraplength=700, font=("Segoe UI", 8, "italic")).pack(anchor="w", padx=10, pady=(0, 10))

    for item in filtered_context:
        q_text = item[0]
        extracted = set(kw for kw, _, _ in extract_keywords(q_text))
        base_keywords = set(kw for kw, _, _ in keywords)
        shared = len(extracted & base_keywords)
        color = "#ff6b6b" if shared <= 1 else "#ffb347" if shared <= 3 else "#7CFC00"
        tk.Label(
            lbl_container,
            text=q_text[:100] + ("..." if len(q_text) > 100 else ""),
            fg=color, bg="#323232", wraplength=700
        ).pack(anchor="w", padx=10)

    # === Onglet 3 : Carte mentale ===
    mindmap_frame = ttk.Frame(notebook, style="TFrame")
    notebook.add(mindmap_frame, text="Carte mentale")

    canvas = tk.Canvas(mindmap_frame, bg="#323232", highlightthickness=0)
    canvas.pack(fill=tk.BOTH, expand=True)

    questions_list = [item[0] for item in filtered_context]
    keywords_per_question = [set(kw for kw, _, _ in extract_keywords(q)) for q in questions_list]

    import math
    n = len(questions_list)
    center_x, center_y = 350, 300
    radius = 250
    nodes_positions = []

    for i in range(n):
        angle = 2 * math.pi * i / n
        x = center_x + radius * math.cos(angle)
        y = center_y + radius * math.sin(angle)
        nodes_positions.append((x, y))

    for i in range(n):
        for j in range(i + 1, n):
            if keywords_per_question[i] & keywords_per_question[j]:
                x1, y1 = nodes_positions[i]
                x2, y2 = nodes_positions[j]
                canvas.create_line(x1, y1, x2, y2, fill="#7CFC00", width=1)

    radius_node = 15
    _tooltip = None

    def show_tooltip(event, text):
        nonlocal _tooltip
        if _tooltip:
            canvas.delete(_tooltip)
            _tooltip = None
        x, y = event.x + 10, event.y + 10
        _tooltip = canvas.create_text(x, y, text=text, anchor="nw", fill="white", font=("Segoe UI", 9), width=300, tags="tooltip")

    def hide_tooltip(event):
        nonlocal _tooltip
        if _tooltip:
            canvas.delete(_tooltip)
            _tooltip = None

    for i, (x, y) in enumerate(nodes_positions):
        node = canvas.create_oval(x - radius_node, y - radius_node, x + radius_node, y + radius_node, fill="#599258", outline="")
        canvas.tag_bind(node, "<Enter>", lambda e, q=questions_list[i]: show_tooltip(e, q))
        canvas.tag_bind(node, "<Leave>", hide_tooltip)

# Barre de statut ET boutons - Frame horizontal unifié
status_buttons_frame = ttk.Frame(main_frame, style='TFrame')
status_buttons_frame.pack(fill=tk.X, pady=(5, 2))

# Statut à gauche
label_status = ttk.Label(
    status_buttons_frame,
    text="Prêt",
    style='Status.TLabel',
    foreground='white',
    anchor='w'  # Alignement à gauche plus naturel
)
label_status.pack(side=tk.LEFT, anchor='w')

# Boutons à droite - dans le même frame horizontal
right_buttons = ttk.Frame(status_buttons_frame, style='TFrame')
right_buttons.pack(side=tk.RIGHT, anchor='e')

btn_info = ttk.Button(right_buttons, text="Détails", style='Bottom.TButton', command=show_infos, width=8)
btn_info.pack(side=tk.TOP, pady=(0, 3))

btn_help = ttk.Button(right_buttons, text="Aide", style='Bottom.TButton', command=show_help, width=8)
btn_help.pack(side=tk.TOP)

# Footer - inchangé
footer_frame = ttk.Frame(root, style='TFrame')
footer_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=(0, 5))

dev_label = ttk.Label(footer_frame, text="Développé par Victor Carré —", style='TLabel', font=('Segoe UI', 8))
dev_label.pack(side=tk.LEFT)

github_link = ttk.Label(footer_frame, text="GitHub", style='Blue.TLabel', cursor="hand2")
github_link.pack(side=tk.LEFT)
github_link.bind("<Button-1>", open_github)

root.mainloop()

: 

# Idées

- Mettre le choix du modèle dans le fichier `config.json`
- Remplacer le hashage par du MD5
- Colorer les contextes en fonction du nombre de keywords communs
- Affichage d'un nuage de mots dans une seconde fenêtre

In [10]:
import torch
print(torch.__version__)

test = "La photocatalyse est une méthode qui permet d'activer des réactions chimiques via la lumière. Elle est utilisée dans l'industrie pharmaceutique pour..."
output = summarizer(test, max_length=150, min_length=75)
print(output)


Your max_length is set to 150, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


2.7.1


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'summary_text': "La photocatalyse est une méthode qui permet d'activer des réactions chimiques via la lumière. Elle est utilisée notamment dans l'industrie pharmaceutique pour faire des économies d'énergie et d'argent, selon une étude de la revue scientifique Plos Medecine, citée par le quotidien américain The Lancet dans son rapport annuel publié mardi.Le site de l'institut."}]
